In [6]:
# creating the classes
from pynput import keyboard as kb

class HauntedMansion:
    def __init__(self):
        self.rooms = {}
        self.doors = {}

    def add_room(self, *rooms):
        for room in rooms:
            self.rooms[room.name] = room

    def add_door(self, *doors):
        for door in doors:
            if door.room1.name in self.rooms and door.room2.name in self.rooms:
                self.doors[door.name] = door
                door.room1.add_door(door)
                door.room2.add_door(door)

class Room:
    def __init__(self, name, description, image=None) :
        self.name = name
        self.description = description
        self.objects = {}
        self.doors = []
        self.image = image

    def add_door(self, door):
         if door not in self.doors:
             self.doors.append(door)

    def add_object(self, *objs):
        for obj in objs:
            self.objects[obj.name] = obj

    def search(self):
        if self.objects:
            print("\nYou found:")
            for obj in self.objects.values():
                print(f"- {obj.name}: {obj.description}")
        else:
            print("\nThere's nothing interesting here.")
        if self.doors:
             print(f"\nYou also find that there are {len(self.doors)} doors:")
             for door in self.doors:
                print(f"- {door.name}")
        else:
             print("\nThere are no doors here.")

class Object:
    def __init__(self, name, description, item=None, key_required=None, interaction_message=None):
        self.name = name
        self.description = description
        self.item = item
        self.key_required = key_required
        self.interaction_message = interaction_message

    def interact(self, player):
        if self.item:
            if self.interaction_message:
                print(self.interaction_message)
            else:
                print(f"You found a {self.item.name} inside the {self.name}!")
            player.inventory.append(self.item)
            self.item = None  # The item is taken
        else:
            print(f"The {self.name} is empty.")

class Locked(Object):
    def __init__(self, name, description, item = None, is_locked=True, correct_pin = None):
        super().__init__(name, description, item)
        self.is_locked = is_locked
        self.correct_pin = correct_pin 
    
    def interact(self, player):
        if self.is_locked:
            print(f"The {self.name} is locked, try to open it!")
            while True:
                pin = input()
                if pin.lower() == "cancel":
                    print("You decided not to try the PIN code.")
                    return  
                try:
                    if len(pin) != 4 or not pin.isdigit():  # check length and if cod contains digits
                        raise ValueError
                    if int(pin) == self.correct_pin:  # check if the pin is correct
                        self.is_locked = False
                        print(f"You unlocked the {self.name}!")
                        super().interact(player)  # open the chest 
                        return  
                    else:
                        print("Incorrect PIN. Try again or type 'cancel':")
                except ValueError:
                    print("Invalid input. Please enter a 4-digit PIN or type 'cancel':")

        else: 
            print("It looks like someone already opened it. There's nothing interesting here.")
    

class Item:
    def __init__(self, name, description):
        self.name = name
        self.description = description

class Door:
    def __init__(self, name, room1, room2, key_required = None):
        self.name = name
        self.room1 = room1
        self.room2 = room2
        self.key_required = key_required

    def get_other_room(self, current_room):
        return self.room2 if current_room == self.room1 else self.room1

class Player:
    def __init__(self):
        self.current_room = None
        self.inventory = []
        self.current_action = None
        self.running = True


    def game_loop(self):
        instructions = "Use 'W' to move, 'Q' to search, 'E' to interact, 'Esc' to quit."
        further_instructions = "To move through a door or interact with an object, please press the corresponding number of your choice."

        print("Welcome to the Haunted Mansion Game. We hope you enjoy playing!")
        print(instructions + " " + further_instructions)
        print("Use 'H' to display instructions again at any point.\n\n")
        print("You wake up and you find yourself in a large room with nothing much in it. The window is broken and you feel the wind blasting inside.\nThe creaking from the old wood around you makes you shiver.")

        def on_press(key):
            try:
                if self.current_action is None:
                    if key.char == "h":  # Show instructions
                        print("\n--- Instructions ---")
                        print(instructions)
                        print("--------------------")
                    elif key.char == "w":
                        self.move()
                    elif key.char == "q":
                        self.search_room()
                    elif key.char == "e":
                        self.interact_with_object()

                elif self.current_action == "moving" and key.char.isdigit():
                    self.handle_move(int(key.char))

                elif self.current_action == "interacting" and key.char.isdigit():
                    self.handle_interaction(int(key.char))

            except AttributeError:
                if key == kb.Key.esc:
                    if self.current_action is None:
                        print("Exiting game...")
                        self.running = False  # Stop the game loop
                        return False  # Stops listener
                    else:
                        print("Exiting action...")
                        self.current_action = None

        with kb.Listener(on_press=on_press) as listener:
            while self.running:
                pass

        listener.stop()

        print("Game loop ended.")

    def move(self):
        if self.current_action:
            return
        
        self.current_action = "moving"
        doors = self.current_room.doors
        if not doors:
            print("\nThere are no doors here. How did you even get here?")
            self.current_action = None
            return
        
        print("\nWhere do you want to move?")
        print("Press a number to select a door or 'Esc' to cancel.\n")
        for index, door in enumerate(doors, start=1):
            print(f"{index} - {door.name}")


    def handle_move(self, index):
        if self.current_action != "moving":
            return
        
        doors = self.current_room.doors
        if 1 <= index <= len(doors):
            selected_door = doors[index - 1]
            other_room = selected_door.get_other_room(self.current_room)
            if selected_door.key_required and selected_door.key_required not in self.inventory:
                print(f"\nYou need a {selected_door.key_required.name} to go there.")
            else:
                self.current_room = other_room
                print(f"\nYou moved to {other_room.name}. {other_room.description}")
                
                if self.current_room.name == "Outside":
                    print("\nCongratulations! You've escaped the haunted mansion!")
                    self.running = False  # Stop the game loop
                    return False

        self.current_action = None  # Exit moving mode
    

    def search_room(self):
        self.current_room.search()


    def interact_with_object(self):
        if self.current_action:
            return
        
        self.current_action = "interacting"
        objects = self.current_room.objects
        if not objects:
            print("\nThere is nothing to interact with here.")
            self.current_action = None
            return

        print("\nWhat do you want to interact with?")
        print("Press a number to interact or 'Esc' to cancel.\n")
        for index, obj in enumerate(objects.values(), start=1):
            print(f"{index} - {obj.name}")
        

    def handle_interaction(self, index):
        if self.current_action != "interacting":
            return
        
        objects = self.current_room.objects
        if 1 <= index <= len(objects):
            obj = list(objects.values())[index - 1]
            if obj.key_required and obj.key_required not in self.inventory:
                print(f"\nYou need another item to access this Object. Maybe look somewhere else?")
            else:
                obj.interact(self)

        self.current_action = None  # Exit interaction mode

In [7]:
# creating & setting the game

# Create the House
house = HauntedMansion()


# Create Rooms
# Ground Floor
outside = Room("Outside", "A breeze of fresh air and singing birds.", "imgs/outside.png")
kitchen = Room("Kitchen", "A warm kitchen with a smell of fresh bread.", "imgs/kitchen.png")
dining_room = Room("Dining Room", "A nicely decorated room with a big table to have a meal.", "imgs/dining.png")
living_room = Room("Living Room", "A huge living room. You hear some songs from the radio.", "imgs/living_room.png")
wc = Room("WC", "A small room with a toilet and a sink.", "imgs/wc.png")
hallway_ground_floor = Room("Hallway (GF)", "An empty hallway on the ground floor.", "imgs/hallway.png")
# First Floor
master_bedroom = Room("Master Bedroom", "A peaceful big bedroom with a soft king-size bed.", "imgs/bedroom.png")
little_guest_room = Room("Little guest Room", "A little room for guests.", "imgs/little_guest_room.png")
big_guest_room = Room("Big guest Room", "A big room for guests.", "imgs/big_guest_room.png")
hallway_first_floor = Room("Hallway (1F)", "An empty hallway on the first floor.", "imgs/hallway1.png")
small_bathroom = Room("Small Bathroom", "A bathroom for guests with a bathtub and a shower.", "imgs/bath.png")
master_bathroom = Room("Master Bathroom", "A big cold bathroom with a toilet and a sink.", "imgs/bathroom.png")
# Keys
silver_key = Item("Silver key", "Opens the door between the dining room and the kitchen")
golden_key = Item("Golden key", "Opens the door between the hallway and the living room")
bronze_key = Item("Bronze key", "Opens the door between the first floor hallway and the small guest room")
platin_key = Item("Platin key", "Opens the door between the first floor hallway and the bedroom")
master_key = Item("Master key", "Opens the big door to the outside")
# Items
hammer = Item("Hammer", "A hammer to destroy the mirror")
chest_code_1 = Item("Chest Code: Part 1", "A piece of paper that contains the first two digits of the code needed to open the chest")
chest_code_2 = Item("Chest Code: Part 2", "A piece of paper that contains the last two digits of the code needed to open the chest")
diary = Item("Diary", "An old diary of someone who lived here in the past. It contains a code")
screwdriver = Item("Screwdriver", "A screwdriver that can be used to open the radio")


# Add Rooms to the House
house.add_room(kitchen, hallway_ground_floor, master_bedroom, dining_room, outside, living_room, wc,
        little_guest_room, big_guest_room, hallway_first_floor, small_bathroom, master_bathroom)


# Create Doors
# Ground Floor
silver_door = Door("Silver door", dining_room, kitchen, silver_key)
white_door = Door("White door", dining_room, hallway_ground_floor)
gold_door = Door("Gold door", hallway_ground_floor, living_room, golden_key)
big_door = Door("Big door", hallway_ground_floor, outside, master_key)
green_door = Door("Green door", hallway_ground_floor, wc)
stairs = Door("Stairs", hallway_ground_floor, hallway_first_floor)
# First Floor
bronze_door = Door("Bronze door", hallway_first_floor, little_guest_room, bronze_key)
platin_door = Door("Platin door", hallway_first_floor, master_bedroom, platin_key)
black_door = Door("Black door", hallway_first_floor, small_bathroom)
red_door = Door("Red door", hallway_first_floor, big_guest_room)
yellow_door = Door("Yellow door", master_bedroom, master_bathroom)


# Connect rooms with doors
house.add_door(silver_door, white_door, gold_door, big_door, green_door, stairs,
        bronze_door, platin_door, black_door, red_door, yellow_door)


# Create Objects
# Ground Floor
chest = Locked("Chest", "A mysterious chest.", golden_key, correct_pin=1920)
cupboard = Object("Cupboard", "A brown wooden cupboard.", chest_code_2, interaction_message="\nYou found a piece of paper with 2 digits written on it. They are '1' and '9'.")
drawer = Object("Drawer", "An old wooden drawer.", chest_code_1, interaction_message="\nYou found a piece of paper with 2 digits written on it. They are '2' and '0'.")
cabinet = Object("Cabinet", "A huge black cabinet.")
radio = Object("Radio", "An ancient radio.", master_key, screwdriver, interaction_message="You can remove the backplate of the radio with the screwdriver you've found earlier. There is a master key inside!\nCan we finally leave this creepy place?")
# First Floor
mirror = Object("Mirror", "A big dirty mirror with a golden frame.", platin_key, hammer, interaction_message="\nYou've destroyed the mirror and found a platinum key in between the shards!")
little_drawer = Object("Little drawer", "A little drawer.", hammer)
desk = Object("Desk", "A wooden desk.", bronze_key, interaction_message="\nYou find a bronze key on the desk. Will it open a door?")
bed = Object("Bed", "Just a messy bed. You woke up here.")
king_bed = Object("King bed", "A huge, dusty bed with unkempt sheets.", diary, interaction_message="\nYou find a diary under the pillow. You open it and see the number '2025' written on the first page. What is it for?")
safe = Locked("Safe", "A safe with a digital lock.", silver_key, correct_pin=2025)
bath_drawer = Object("Bathroom Drawer", "A drawer below the sink.", screwdriver, interaction_message="After opening the drawer, you find a screwdriver inside. Maybe it is useful.")

# Add Objcts to rooms
dining_room.add_object(drawer)
kitchen.add_object(chest, cupboard)
little_guest_room.add_object(little_drawer, mirror)
big_guest_room.add_object(bed, desk)
master_bedroom.add_object(king_bed, safe)
living_room.add_object(radio, cabinet)
master_bathroom.add_object(bath_drawer)


# Create a player and set their starting room
player = Player()
player.current_room = big_guest_room

In [8]:
player.game_loop()

This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.



You are in Big guest Room. A big room for guests.
Use 'W' to move, 'Q' to search, 'E' to interact, 'Esc' to quit. To move through a door or interact with an object, please press the corresponding number of your choice.
Use 'H' to display instructions again at any point.
You found:
- Bed: A messy bed. You woke up here!
- Desk: A wooden desk.
You also find that there are 1 doors:
- Red door


KeyboardInterrupt: 